# Calculator for common DSA-2000 Questions

## Casey Law, claw@astro.caltech.edu

Written in support of DSA-2000 science workshop (January 2022). Reproduces work by Steve Myers for VLASS and DSA-2000 survey design.

In [1]:
%matplotlib inline

import pylab as pl
import numpy as np
import healpy

## Define some useful functions


In [2]:
resolution = lambda freq, bmax: 1.22*3600*np.degrees(3e8/float(freq)/bmax) # resolution in asec for freq in Hz, bmax in meters
sensitivity = lambda sefd, dt, bw, eta, nbl: sefd/(eta*np.sqrt(nbl*2 * dt * bw * 2))    # sefd in Jy, dt in s, bw in Hz, assumes std correlator eficiency and 2 pols
tobs = lambda s0, sefd, na, bw, eta: (sefd/(s0*eta))**2/(na*(na-1)*bw*2)    # obsering time in s (inverse of sensitivity eqn)
area_field = lambda fov: 0.5665 * fov**2  # fov in amin
surveyspeed = lambda fov, t0: area_field(fov)/t0         # ss in deg2/hr, fov in amin, t0 is time per pointing in s
f_beam = lambda rad, fwhm: np.exp(-4*np.log(2)*(rad/fwhm)**2)  # beam function (fractional power at radius rad)

datarate_vis = lambda na, nch, npol, tsamp: (na*(na-1)/2)*nch*npol*8/tsamp/1024**2  # MB/s for tsamp in s
datarate_vis_vla = lambda tsamp, nspw: 45*(nspw*64*4/16384.) / tsamp # scaling taken from nrao oss, minimal nchan, full pol

In [3]:
ess_nu = lambda fov, tobs, nu, alpha: surveyspeed(fov, tobs) * nu**(2*alpha)
ess_flux = lambda fov, tobs, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2)
ess_nuflux = lambda fov, tobs, nu, alpha, sensitivity, gamma: surveyspeed(fov, tobs) * sensitivity**(gamma+2) * nu**(2*alpha)
surveytime = lambda band, sensitivity, area, overhead: overhead * area/surveyspeed(fov[band], tobs(sensitivity, sefd[band], na[band], bw_max[band], eta[band]))   # total time in hours for given band, area in sq deg and sensitivity in Jy

# Effective survey speed for extragalactic specs (alpha = -0.7, gamma => euclidean).
ess_eg = lambda fov, dt, nu, sensitivity: ess_nuflux(fov, dt, nu, -0.7, sensitivity, -1.5)
sigma_confusion = lambda nu, theta: 1.2e-6*(nu/3.02e9)**(-0.7)*(theta/8)**(10/3)  # confusion with freq in Hz, theta in asec
survey_area = lambda dec_lim: (1+np.sin(np.radians(np.abs(dec_lim))))*(2*np.pi*(180/np.pi)**2)  # sq deg, exclude outside dec_lim

In [4]:
def find_hpnside(size, verbose=False):
    """ Use healpix scheme to find optimal nsize for a given size in degrees
    """
    for i in range(20):
        resol = np.degrees(healpy.nside2resol(2**i))
        if resol < size:
            if verbose:
                print(f'nside=2**{i} has resolution of {resol:1.1e} deg (resolves limit of {size} deg)')
            break
    return 2**i

## Set telescope parameters as a function of band/config

In [5]:
# keys defined for configurations and bands separately.
names = ['DSA', 'VLA', 'ASKAP']
na = {'DSA': 2000, 'VLA': 27, 'ASKAP': 36}   # number of antennas
freq = {'DSA': 1.35e9, 'VLA': 3.0e9, 'ASKAP': 1.28e9}  # band center
fov = {'DSA': 60*3.06, 'VLA': 15., 'ASKAP': 60*3.}    # fwhm field of view (theta_pb) in arcminutes
bw_max = {'DSA': 0.65*1.3e9, 'VLA': 1.5e9, 'ASKAP': 288e6}    # bandwidth (minus RFI)
bmax = {'DSA': 17e3, 'VLAB': 11.1e3, 'ASKAP': 6e3}    # longest baselines in meter
sefd = {'DSA': 5020, 'VLA': 370, 'ASKAP': 1952}    # array SEFD
# VLA sefd values appear high, because we assume 27 ants, whereas exposure calculator uses 25 ants. result is the same in both cases.
eta = {'DSA': 1.0, 'VLA': 0.92, 'ASKAP': 1.0}   # correlator efficiency
surveyspecs = {'DSA All-Sky': [3e4, 2.04e-6, 1.25, 1.0],
               'VLASS': [3.3885e4, 122e-6, 1.17, 1.09],
               'ASKAP EMU': [3.0939e4, 10e-6, 1.25, 1.0]}  # something wrong with ASKAP numbers

# Q&A

### Given sensitivity/area, what is the cadence?

In [6]:
s0 = 2e-6
a0 = 3e4
o0 = 1.25
t_total = 5 * 365.24 * 24  # 5 years in hrs
frac_survey = 0.65
t_epoch = surveytime('DSA', s0, a0, o0)
print(f'{t_epoch:.0f} hrs to cover {a0:.0f} sq deg to depth of {s0:.1e} Jy with {frac_survey*t_total/t_epoch:.1f} epochs in {t_total/(24*365.24):.1f} years')

1831 hrs to cover 30000 sq deg to depth of 2.0e-06 Jy with 15.6 epochs in 5.0 years


### And for deep field?

In [7]:
s0 = 100e-9
a0 = 30.
o0 = 1.25
t_total = 5 * 365.24 * 24  # 5 years in hrs
frac_deep = 0.05
t_survey = surveytime('DSA', s0, a0, o0)
print(f'{t_survey:.0f} hrs to cover {a0:.0f} sq deg to depth of {s0:.1e} Jy with {frac_deep*t_total/t_epoch:.1f} epochs in {t_total/(24*365.24):.1f} years')

732 hrs to cover 30 sq deg to depth of 1.0e-07 Jy with 1.2 epochs in 5.0 years


### Given cadence/area, what is the sensitivity?

In [8]:
# this one seems off

band = 'DSA'
a0 = 3e4
t_total = 5*365.24*24 # 5 years in hrs
o0 = 1.25
cadence = 1/(24*30)

n_fields = a0/area_field(fov[band]/60)
t_field = cadence*t_total/n_fields

print(f'{n_fields:.0f} fields (of {area_field(fov[band]/60):.1f} deg^2) of depth {sensitivity(sefd[band], t_field/o0, bw_max[band], eta[band], (na[band]*(na[band]-1)/2)):.1e} Jy')

5656 fields (of 5.3 deg^2) of depth 6.6e-04 Jy


### How long until reaching the confusion limit?

In [9]:
theta0 = resolution(freq['DSA'], bmax['DSA'])
sigma_c0 = sigma_confusion(freq['DSA'], theta0)
t0 = tobs(sigma_c0, sefd['DSA'], na['DSA'], bw_max['DSA'], eta['DSA'])

print(f"Confusion limit of {1e9*sigma_c0:.0f} nJy reached in {t0/3600:.1f} hr ({theta0:0.1f} arcsec beam at {freq['DSA']/1e9} GHz)")


Confusion limit of 109 nJy reached in 87.2 hr (3.3 arcsec beam at 1.35 GHz)


### Which deep fields are best?

In [10]:
# TBD: time to cover existing deep fields
field_properties = {'COSMOS': [], 'HDFS': [], 'HUDF': [], 'CDFS': []}


### How many radio continuum sources detected?

In [11]:
s0 = 2e-6
a0 = 3e4

#n_source = lambda s_lim, resolution: ...

### How many spectral points can be measured per source?

### What minimum declination is best for all-sky survey?

In [12]:
dec_min = -30
area30 = survey_area(dec_min)

# For elliptical array (17 km N-S vs 15 km E-W baselines), calc southerly zenith angle for circular beam
za0 = np.degrees(np.cos(15/17))
lat = +39  # rough latitude of array
beam_elong = np.cos(np.radians((lat-za0)-dec_min))
print(f'At min Dec of {dec_min}, the survey area is {area30:.0f} and transit beam elongation is {beam_elong:0.2f}')

At min Dec of -30, the survey area is 30940 and transit beam elongation is 0.84


### How many pointings to cover the sky?

In [13]:
dec_min = -30
nside_thetap = find_hpnside(fov['DSA']/60)
n_pointings = int(np.round(healpy.nside2npix(nside_thetap)*(survey_area(dec_min)/survey_area(-90))))
thetap_area = healpy.nside2pixarea(nside_thetap, degrees=True)
print(f'Cover the sky with {n_pointings} pointings of area {thetap_area:.2f} deg^2')

Cover the sky with 9216 pointings of area 3.36 deg^2


### What is relative sensitivity at top/bottom of band?

### How big is all-sky image?

In [14]:
oversample = 4  # pixel/beam
nside_thetab = find_hpnside(resolution(freq['DSA'], bmax['DSA'])/3600/oversample)
n_pix_tot = int(np.round(healpy.nside2npix(nside_thetab)*(survey_area(dec_min)/survey_area(-90))))
thetab_resol = healpy.nside2resol(nside_thetab, arcmin=True)*60
print(f'Pixelate the sky with {n_pix_tot/1024**4} Tpixels of resolution {thetab_resol:.2f} arcsec (healpix nside={nside_thetab})')
print(f'\tThis requires {8*n_pix_tot/1024**4} TB (Stokes I, one band)')

print(f'{nside_thetab//nside_thetap} pixels per pointing (thetab side length)')
print(f'{int(2*1.19*nside_thetab//nside_thetap)} pixels per pointing (first null side length)')

Pixelate the sky with 0.5625 Tpixels of resolution 0.81 arcsec (healpix nside=262144)
	This requires 4.5 TB (Stokes I, one band)
8192 pixels per pointing (thetab side length)
19496 pixels per pointing (first null side length)


### Still working this in...

* Given channel bandwidth, what is sensitivity channel (HI)? (Tb sensitivity; scale as input parameter on cumulative baseline distribution; just change bandwidth and add scale)

# More General Comparisons

## Telescope performance

In [15]:
# somethign wrong with ASKAP numbers

print(f'Array \t   Tobs (s) \t    SS (OTF; deg2/hr) \t Sens. (Jy) \t hrs/3e4 deg2 (ideal)')
print('----------------------------------------------------------------------------------------------')
for name in names:
    na0 = na[name]
    surveykey = list(filter(lambda x: name in x, surveyspecs.keys()))[0]
    s0 = surveyspecs[surveykey][1]
    t0 = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
    s0 = sensitivity(sefd[name], t0, bw_max[name], eta[name], na0*(na0-1)/2)
    print(f'{name} \t   {t0:.1f}  \t\t {surveyspeed(fov[name], t0):.1f}   \t {s0:1.2e} \t {surveytime(name, s0, 30000, 1.0):.1f} ')

Array 	   Tobs (s) 	    SS (OTF; deg2/hr) 	 Sens. (Jy) 	 hrs/3e4 deg2 (ideal)
----------------------------------------------------------------------------------------------
DSA 	   896.2  		 21.3   	 2.04e-06 	 1408.0 
VLA 	   5.2  		 24.7   	 1.22e-04 	 1214.5 
ASKAP 	   52500.9  		 0.3   	 1.00e-05 	 85811.0 


## Time required for surveys

In [16]:
def print_survey(surveyspecs):
    ss = 'Survey    \t s_lim   \t SS     \t Time/pointing \t Total Time\n'
    ss += '          \t (Jy)    \t (deg/hr)  \t (s)    \t   (hr)\n'
#    ss +='--------------------------------------------------------------------\n'
    for name in names:
        surveykey = list(filter(lambda x: name in x, surveyspecs.keys()))[0]
        specs = surveyspecs[surveykey]
        a0, s0, o0, tsysfac = specs
        na0 = na[name]
        tint = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
        t0_pointing = tobs(s0, sefd[name], na0, bw_max[name], eta[name])
        t0_survey = tsysfac*surveytime(name, s0, a0, o0)
        surveyspeed0 = surveyspeed(fov[name], t0_pointing)
        ss += f'{surveykey}    \t {s0:.2e} \t {surveyspeed0:.1f}    \t {t0_pointing:.1f}    \t {t0_survey:.1f}\n'
    print(ss)

print_survey(surveyspecs)

Survey    	 s_lim   	 SS     	 Time/pointing 	 Total Time
          	 (Jy)    	 (deg/hr)  	 (s)    	   (hr)
DSA All-Sky    	 2.04e-06 	 21.3    	 896.2    	 1760.0
VLASS    	 1.22e-04 	 24.7    	 5.2    	 1749.4
ASKAP EMU    	 1.00e-05 	 0.3    	 52500.9    	 110621.1



## Other DSA-2000 surveys/products

- Spectroscopy
-- 9700 channels at 134kHz resolution
-- 4700 channels at 8.4kHz (HI/OH)
- Pulsars/FRBs
-- 16384 channels, 0.8 ms sampling with 20 seconds triggers => 3 GB/beam/trigger
-- 1 s dedispersed voltage cutout per candidate, 2e9 samples/s => 64 TB/trigger 
- Polarimetry
-- 1000 channels (1.3 MHz, Stokes QUV)